In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
import os

img_size = (224, 224)
batch_size = 32

train_dir = "/content/drive/MyDrive/Data/Teeth_Dataset/Training"
val_dir   = "/content/drive/MyDrive/Data/Teeth_Dataset/Validation"
test_dir  = "/content/drive/MyDrive/Data/Teeth_Dataset/Testing"


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create generator with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.15,
    horizontal_flip=True,
    brightness_range=(0.8,1.2),
    fill_mode='nearest'
)


val_datagen = ImageDataGenerator(rescale=1./255)  # only rescale for validation

# Apply to directories
train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


Found 3087 images belonging to 7 classes.
Found 1028 images belonging to 7 classes.
Found 1028 images belonging to 7 classes.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling2D, Flatten, Dense, BatchNormalization, Activation, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.metrics import Precision, Recall
from tensorflow.keras.optimizers import Adam
from tensorflow import keras



model = Sequential()

#Feature extraction
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

model = Sequential()

# Block 1
model.add(Conv2D(16, (3,3), activation='relu', input_shape=(224,224,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))

# Block 2
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))

# Block 3
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))

# Flatten + Classifier
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dense(7, activation='softmax'))


optimizer = Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss='categorical_crossentropy',  metrics=['accuracy', Precision(), Recall()])
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_5 (Conv2D)               │ (None, 222, 222, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 222, 222, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 111, 111, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 109, 109, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 109, 109, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 52, 52, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 52, 52, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 26, 26, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 43264)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 43264)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     5,537,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,562,855 (21.22 MB)

 Trainable params: 5,562,631 (21.22 MB)

 Non-trainable params: 224 (896.00 B)

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau, LearningRateScheduler
callbacks = [
            EarlyStopping(patience = 8, verbose=1),
            ReduceLROnPlateau(factor = 0.5, patience = 5, min_lr = 0.000001, verbose=1),
            ModelCheckpoint('model_Teeth.keras', monitor='val_loss', save_best_only=True, verbose=1)
            ]

hist = model.fit(
    train_gen,
    validation_data=val_gen,
    shuffle=True,
    epochs=80,
    callbacks=callbacks,
)

Epoch 1/80
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.2957 - loss: 3.4239 - precision: 0.3258 - recall: 0.2052
Epoch 1: val_loss improved from inf to 17.38563, saving model to model_Teeth.keras
97/97 ━━━━━━━━━━━━━━━━━━━━ 1117s 11s/step - accuracy: 0.2960 - loss: 3.4135 - precision: 0.3265 - recall: 0.2049 - val_accuracy: 0.1449 - val_loss: 17.3856 - val_precision: 0.1449 - val_recall: 0.1449 - learning_rate: 5.0000e-04
Epoch 2/80
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 516ms/step - accuracy: 0.4436 - loss: 1.4537 - precision: 0.6698 - recall: 0.2379
Epoch 2: val_loss did not improve from 17.38563
97/97 ━━━━━━━━━━━━━━━━━━━━ 54s 554ms/step - accuracy: 0.4436 - loss: 1.4539 - precision: 0.6696 - recall: 0.2380 - val_accuracy: 0.1449 - val_loss: 19.2064 - val_precision: 0.1449 - val_recall: 0.1449 - learning_rate: 5.0000e-04
Epoch 3/80
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 514ms/step - accuracy: 0.4638 - loss: 1.4179 - precision: 0.6871 - recall: 0.2656
Epoch 3: val_loss did not improve from 17.385